In [1]:
### Colab define cell
FOLDER_PATH = '/content/gdrive/MyDrive/Counterfactual-prototype-main/LORE/LORE-master'

try:
    import google.colab
    RUNNING_ON_COLAB = True
except: 
    RUNNING_ON_COLAB = False

if RUNNING_ON_COLAB: 
    import sys
    from google.colab import drive
    from pydrive.auth import GoogleAuth
    from pydrive.drive import GoogleDrive
    from google.colab import auth
    from oauth2client.client import GoogleCredentials
    from pydrive.auth import GoogleAuth
    from pydrive.drive import GoogleDrive
    from google.colab import auth
    from oauth2client.client import GoogleCredentials 
    drive.mount('/content/gdrive', force_remount=True)
    auth.authenticate_user()
    gauth = GoogleAuth()
    gauth.credentials = GoogleCredentials.get_application_default()
    gdrive = GoogleDrive(gauth)
    sys.path.append(FOLDER_PATH)

In [2]:
from utils.prediction import PredictionTypeWrapper, PredictionTypeWrapper, generate_local_predictions, wrap_information
from lore import LOREWrapper
from lore.prepare_dataset import prepare_diabetes_dataset
from lore.evaluation import evaluate_explanation
import lore
from utils import load

import tensorflow as tf
import numpy as np
import pandas as pd

from lore.prepare_dataset import *
from lore.neighbor_generator import *

### Set random seed
seed = 123
tf.random.set_seed(seed)
np.random.seed(seed)

2021-07-18 16:29:51.040116: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0


In [3]:
### Load data
dataset = load.SelectableDataset.Diabetes
data, balanced_data, X, Y, encoder, scaler, n_features, n_classes, feature_names, target_name = load.load_dataset(dataset)
X_train, Y_train, X_test, Y_test, X_validation, Y_validation = load.load_training_data(dataset)
model = load.load_trained_model_for_dataset(dataset)

Loaded model from disk


2021-07-18 16:29:51.974030: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcuda.so.1
2021-07-18 16:29:52.006010: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-07-18 16:29:52.006348: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: NVIDIA GeForce GTX 1070 computeCapability: 6.1
coreClock: 1.695GHz coreCount: 16 deviceMemorySize: 7.92GiB deviceMemoryBandwidth: 238.66GiB/s
2021-07-18 16:29:52.006377: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
2021-07-18 16:29:52.009173: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcublas.so.11
2021-07-18 16:29:52.009222: I tensorflow/stream_executor/pla

In [4]:

diabetes_feature_range = (X_train.min(axis=0), X_train.max(axis=0))
# store all information 
local_data_dict = generate_local_predictions( X_test, Y_test, model, scaler, encoder )
# sorting by different conditions
true_positives,true_negatives, false_positives, false_negatives = wrap_information( local_data_dict )
# get all prediction types
all_predictions = PredictionTypeWrapper(true_positives,true_negatives, false_positives, false_negatives)

2021-07-18 16:29:52.681632: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:176] None of the MLIR Optimization Passes are enabled (registered 2)
2021-07-18 16:29:52.682094: I tensorflow/core/platform/profile_utils/cpu_utils.cc:114] CPU Frequency: 2199995000 Hz
2021-07-18 16:29:52.754116: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcublas.so.11
2021-07-18 16:29:53.000431: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcublasLt.so.11


In [5]:
lore_wrapper = LOREWrapper.LOREWarpper(model)
diabetes_dataset = prepare_diabetes_dataset('./datasets/diabetes.csv')

In [6]:
X2E = X_test
y2E = lore_wrapper.predict(X2E).flatten()
y2E = np.asarray([diabetes_dataset['possible_outcomes'][i] for i in y2E])

In [7]:
print(diabetes_dataset['label_encoder'][diabetes_dataset['class_name']].classes_)
print(diabetes_dataset['possible_outcomes'])

[0 1]
[1, 0]


In [8]:
idx_record2explain = 0

In [9]:
path_data = './lore/datasets/'
explanation, infos = lore.explain(idx_record2explain, X2E, diabetes_dataset, lore_wrapper,
                                  ng_function=genetic_neighborhood,
                                  discrete_use_probabilities=True,
                                  continuous_function_estimation=False,
                                  returns_infos=True,
                                  path=path_data, sep=';', log=False)

/home/jrhs/.pyenv/versions/3.8.11/envs/tf/lib/python3.8/site-packages/deap/creator.py:138: RuntimeWarning: A class named 'fitness' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
/home/jrhs/.pyenv/versions/3.8.11/envs/tf/lib/python3.8/site-packages/deap/creator.py:138: RuntimeWarning: A class named 'individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "


In [10]:
dfX2E = build_df2explain(lore_wrapper, X2E, diabetes_dataset).to_dict('records')
dfx = dfX2E[idx_record2explain]
# x = build_df2explain(blackbox, X2E[idx_record2explain].reshape(1, -1), dataset).to_dict('records')[0]

print('x = %s' % dfx)
print('r = %s --> %s' % (explanation[0][1], explanation[0][0]))
for delta in explanation[1]:
    print('delta', delta)

covered = lore.get_covered(explanation[0][1], dfX2E, diabetes_dataset)
print(len(covered))
print(covered)

print(explanation[0][0][diabetes_dataset['class_name']], '<<<<')

def eval(x, y):
    return 1 if x == y else 0

precision = [1-eval(v, explanation[0][0][diabetes_dataset['class_name']]) for v in y2E[covered]]
print(precision)
print(np.mean(precision), np.std(precision))


x = {'Outcome': 1, 'Pregnancies': 0, 'Glucose': 0, 'BloodPressure': 0, 'SkinThickness': 0, 'Insulin': 1, 'BMI': 0.4485842026825634, 'DiabetesPedigreeFunction': 0.13404710920770876, 'Age': 0}
r = {'Insulin': '>0', 'DiabetesPedigreeFunction': '>0.105657', 'BMI': '<=0.490295'} --> {'Outcome': 1}
delta {'DiabetesPedigreeFunction': '<=0.155452'}
delta {'DiabetesPedigreeFunction': '>0.220583'}
1
[0]
1 <<<<
[1]
1.0 0.0


In [11]:
x = build_df2explain(lore_wrapper, X2E[idx_record2explain].reshape(1, -1), diabetes_dataset).to_dict('records')[0]

print('x = %s' % x)
print('r = %s --> %s' % (explanation[0][1], explanation[0][0]))
for delta in explanation[1]:
    print('delta', delta)

print('Evaluation')
bb_outcome = infos['bb_outcome']
cc_outcome = infos['cc_outcome']
y_pred_bb = infos['y_pred_bb']
y_pred_cc = infos['y_pred_cc']
dfZ = infos['dfZ']
dt = infos['dt']
tree_path = infos['tree_path']
leaf_nodes = infos['leaf_nodes']
diff_outcome = infos['diff_outcome']

print(evaluate_explanation(x, lore_wrapper, dfZ, dt, tree_path, leaf_nodes, bb_outcome, cc_outcome,
                            y_pred_bb, y_pred_cc, diff_outcome, diabetes_dataset, explanation[1]))

x = {'Outcome': 1, 'Pregnancies': 0, 'Glucose': 0, 'BloodPressure': 0, 'SkinThickness': 0, 'Insulin': 1, 'BMI': 0.4485842026825634, 'DiabetesPedigreeFunction': 0.13404710920770876, 'Age': 0}
r = {'Insulin': '>0', 'DiabetesPedigreeFunction': '>0.105657', 'BMI': '<=0.490295'} --> {'Outcome': 1}
delta {'DiabetesPedigreeFunction': '<=0.155452'}
delta {'DiabetesPedigreeFunction': '>0.220583'}
Evaluation
1,0.821970,0.823308,0.845000,0.915989,4,1,8,0.997500,1.000000
